In [ ]:
import random
import math
import requests
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

%run "source/RSA Helper Functions.ipynb"

To better understand the practical application of the theoretical principles we've discussed, let's walk through the key generation, encryption, and decryption processes involved in the RSA algorithm.

At first, we'll implement a very basic, straightforward algorithm using small numbers. We will then evaluate this algorithm to identify any parts that are causing performance issues or acting as bottlenecks that slow down execution. Based on the testing results and analysis, we'll attempt to optimize and enhance the underperforming sections of the algorithm to improve its overall efficiency.

### 5.1 Trivial Implementation

First let's prepare the functions that will be needed to perform the 5 steps of the key generation process.

#### 5.1.1 Generate prime numbers

In [ ]:
def is_prime (number):
    if number < 2:
        return False
    for i in range (2, number // 2 + 1):
        if number % i == 0:
            return False
    return True

def get_random_number(n_bits):
    """Generates a random number n_bits long"""
    return random.randrange(2**(n_bits-1) + 1, 2**n_bits - 1)

def generate_prime(n_bits):
    """Generates a random prime number n_bits long"""
    while True:
        prime = get_random_number(n_bits)
        if is_prime(prime):
            return prime

#### 5.2.2 Generate $\varphi(n)$

In [ ]:
def eulers_totient(p, q):
    return (p-1) * (q-1)

#### 5.2.3 Generate public exponent $e$

In [ ]:
def generate_public_exponent(phi_n):
    """
    Generates a public exponent 'e' for RSA encryption that is coprime with phi_n.
    Args:
        phi_n (int): The totient of the product of two prime numbers (phi_n = (p-1)*(q-1)).
    Returns:
        int: A public exponent 'e' that is coprime with phi_n.
    """
    e = random.randint (3, phi_n-1)
    while math.gcd(e, phi_n) != 1:
        e = random.randint (3, phi_n - 1)

    return e

#### 5.2.4 Calculate the modular inverse $d$

In [ ]:
def mod_inverse(e, phi):
    """
    Calculates the modular inverse of 'e' modulo 'phi' using brute force.
    Args:
        e (int): The number for which to find the modular inverse.
        phi (int): The modulus.
    Returns:
        int: The modular inverse of 'e' modulo 'phi'.
    """
    for d in range (3, phi):
        if (d * e) % phi == 1:
            return d
    raise ValueError (f"mod_inverse of ({e}, {phi}) does not exist!")

#### 5.2.5 Generate keys

Now let's combine the above functions to generate the public and private keys. The function `generate_keys()` will also return $n$, which will be used in subsequent encryption/decryption.

In [ ]:
def generate_keys(p = None, q = None, e = None, prime_length_bits = 8):
    """
    Generates RSA public and private keys.
    Args:
        p (int, optional): The first prime number. If not provided, it will be generated.
        q (int, optional): The second prime number. If not provided, it will be generated.
        e (int, optional): The public exponent. If not provided, it will be generated.
        prime_length_bits (int, optional): The bit length for generating prime numbers. Default is 256.
    Returns:
        tuple: A tuple containing the public exponent 'e', the private exponent 'd', and the modulus 'n'.
    """
    # Step 1: Generate primes
    if ((not p) or (not q)):
        p, q = generate_prime(prime_length_bits), generate_prime (prime_length_bits)
        if p == q:
            q = generate_prime(prime_length_bits)
    else:
        if (not is_prime(p) or not is_prime(q)):
            raise ValueError (f"p and q must be prime.")

    # Step 2: Generate the product of the primes
    n = p * q

    # Step 3: Generate φ(n)
    phi_n = eulers_totient(p, q)

    # Step 4: Generate the public exponent e
    if (not e):
        e = generate_public_exponent(phi_n)

    # Step 5: Calculate the modular inverse d
    d = mod_inverse(e, phi_n)

    return e, d, n

#### 5.2.6 Encrypt/Decrypt the message

In [ ]:
def encrypt_message_to_cipher(M, e, n):
    """
    Encrypts a string message using the RSA encryption algorith.
    Args:
        M (str): The plaintext message to be encrypted.
        e (int): The public exponent used for encryption.
        n (int): The modulus used for encryption.
    Returns:
        list: A list of integers representing the encrypted message.
    """
    M_ascii_encoded = [ord(ch) for ch in M]
    cipher = [pow(ch, e, n) for ch in M_ascii_encoded]

    return cipher

def decrypt_cipher_to_message(C, d, n):
    """
    Decrypts an encrypted message using the RSA decryption algorithm.
    Args:
        C (list): The encrypted message represented as a list of integers.
        d (int): The private exponent used for decryption.
        n (int): The modulus used for decryption.
    Returns:
        str: The decrypted plaintext message.
    """
    C_ascii_encoded = [pow(ch, d, n) for ch in C]
    M = "".join (chr(ch) for ch in C_ascii_encoded)

    return M

#### 5.2.7 Combining all components

In [ ]:
e, d, n = generate_keys()

message = "A"
encrypted = encrypt_message_to_cipher(message, e, n)
decrypted = decrypt_cipher_to_message(encrypted, d, n)

print("Public Key:  ", e)
print("Private Key: ", d)
print("n:           ", n)
print("Original message:  ", message)
print("Encrypted by RSA   ", encrypted)
print("Decrypted message: ", decrypted)

#### 5.2.8 Functional tests

Now is a good time to write some tests so that we know we haven't broken our implementation with future changes of the code.

In [ ]:
def test_decrypting(message):
    e, d, n = generate_keys()
    encrypted = encrypt_message_to_cipher(message, e, n)
    decrypted = decrypt_cipher_to_message(encrypted, d, n)
    assert decrypted == message, "The decrypted message is not the same as the original message!"

def all_functional_tests():
    text = Path("test_data/file_6000_chars.txt").read_text()
    messages = ["A", "z", "AB", "abc", text[:100], text[:200], text[:500]]
    for message in messages:
        test_decrypting(message)

    print("\033[32m ✅ All tests passed \033[0m")

In [ ]:
all_functional_tests()

### 5.2 Analyzing the Algorithm

In this section, we will explore the impact of modifying various parameters involved in the RSA algorithm, particularly focusing on how these changes affect its performance.

#### 5.2.1 How the size of $p$ and $q$ affects the time for their generation?

##### Explore the time to generate prime numbers of varying lengths using brute force prime generation

Let's explore the time required to generate prime numbers of varying lengths. We will generate specific primes by constraining the range to the desired prime. The current algorithm in function `generate_prime()` we created above (🔴link to section ) employs a brute-force approach, generating a random number within a specified range and then checking its primality using a brute-force method.

In [ ]:
# function `mean_time_msec` is defined in the companion notebook RSA Helper Functions.ipynb
def test_prime_generation_performance(n_bits_list, fn_generate_prime):
    return [mean_time_msec(5)(fn_generate_prime)(n_bits) for n_bits in n_bits_list]

<div class="alert alert-block alert-warning" style="font-size:1.2em">
⚠️The below cell will run for $≈$3 min. If you want to run it set the flag to True. Otherwise you can inspect the output in the screenshot below the cell.
</div>

In [ ]:
I_UNDERSTAND_THE_CELL_WILL_RUN_LONG_TIME = False

if I_UNDERSTAND_THE_CELL_WILL_RUN_LONG_TIME:
    n_bits_list = [5, 10, 20, 25, 27, 30]
    print(test_prime_generation_performance(n_bits_list, generate_prime))

Output of the above cell:

![generate_primes_brute_force.png](images/generate_primes_brute_force.png)

Generating a 30 bit prime number using the brute-force method took $≈$ 33 seconds. Clearly a performance hog! Let's try to improve the performance by using another algorithm like the Sieve of Eratosthenes.

##### Try to improve prime generation by using the Sieve of Eratosthenes

In [ ]:
I_UNDERSTAND_THE_CELL_WILL_RUN_LONG_TIME = False

# functions `generate_prime_sieve_eratosthenes` and `mean_time_msec` are
# defined in the companion notebook RSA Helper Functions.ipynb
if I_UNDERSTAND_THE_CELL_WILL_RUN_LONG_TIME:
    n_bits_list = [5, 10, 20, 25, 27, 30]
    print(test_prime_generation_performance(n_bits_list, generate_prime_sieve_eratosthenes))

Output of the above cell:
![generate_primes_sieve_er.png](images/generate_primes_sieve_er.png)

Generating a 30-bit prime number using the Sieve of Eratosthenes took 221 seconds. This is much worse performance. However, this is expected because the Sieve of Eratosthenes is designed to generate a list of all prime numbers up to a given limit, whereas in this case we only needed one prime number of a specific length. As a result, the algorithm performed more work than was necessary to achieve the desired outcome, leading to the longer execution time.

##### Another improvement attempt by using the Miller-Rabin Primality test

Let's make another attempt to improve the prime generation by using the Miller-Rabin primality test instead of the brute force `is_prime()`. This algorithm can produce false positives that is why it is executed multiple times to increase the probability of a correct result.

In [ ]:
def generate_prime_miller_rabin(n_bits):
    """
    Generates a prime number of the specified bit length using the Miller-Rabin primality test.
    Args:
        n_bits (int): The bit length for the prime number to be generated.
    Returns:
        int: A prime number of the specified bit length.
    """
    while True:
        prime = get_random_number(n_bits)
        if is_prime_miller_rabin(prime, k=5):
            return prime

In [ ]:
I_UNDERSTAND_THE_CELL_WILL_RUN_LONG_TIME = False

if I_UNDERSTAND_THE_CELL_WILL_RUN_LONG_TIME:
    n_bits_list = [256, 516, 1024, 2048, 4096]
    print(test_prime_generation_performance(n_bits_list, generate_prime_miller_rabin))

Output of the above cell:
![generate_primes_miller_rabin.png](images/generate_primes_miller_rabin.png)

We see that even though we saw good times for generation of small and mid-sized primes, even the Miller-Rabin algorithm fails in terms of performance for 2048-bit and larger primes. Let's make one more attempt at improving performance by combining the Sieve or Eratosthenes and the Rabin-Miller primality test. The idea is to generate less prime candidates by first excluding some of the non-primes using the sieve.

##### Yet another attempt to improve performance by combining the Sieve of Eratosthenes and the Rabin-Miller primality test

In [ ]:
# function `is_prime_miller_rabin` is defined in the companion notebook RSA Helper Functions.ipynb
def generate_prime_miller_rabin_sieve_er (n_bits):
    """
    Generates a prime number of the specified bit length using the Sieve of Eratosthenes
    for choosing a prime candidate and the Miller-Rabin primality test.
    Args:
        n_bits (int): The bit length for the prime number to be generated.
    Returns:
        int: A prime number of the specified bit length.
    """
    while True:
        prime_candidate = get_random_number_sieved(n_bits)
        if not is_prime_miller_rabin(prime_candidate, k=5):
            continue
        else:
            return prime_candidate

In [ ]:
I_UNDERSTAND_THE_CELL_WILL_RUN_LONG_TIME = False
# function `is_prime_miller_rabin_sieve_er` is defined in the companion notebook RSA Helper Functions.ipynb

if I_UNDERSTAND_THE_CELL_WILL_RUN_LONG_TIME:
    n_bits_list = [256, 516, 1024, 2048, 4096]
    print(test_prime_generation_performance(n_bits_list, generate_prime_miller_rabin_sieve_er))

Output of the above cell:
![generate_primes_rabin_miller_sieve_er.png](images/generate_primes_rabin_miller_sieve_er.png)


Finally, we have enough information to compare the time for generating prime numbers of different lengths using two algorithms:

In [ ]:
n_bits_list = [256, 516, 1024, 2048, 4096]
times_to_generate_prime_mr = [23.704800001723925, 59.54100000235485, 2779.947099999845, 9221.19549999843, 266755.3584999987]
times_to_generate_prime_mr_sieve = [17.45451996102929, 61.99582004919648, 398.5925999470055, 4936.42343999818, 27441.254119947553]

plt.figure(figsize=(7, 4))
plt.plot(n_bits_list, times_to_generate_prime_mr, marker="o", label="Rabin-Miller")
plt.plot(n_bits_list, times_to_generate_prime_mr_sieve, marker='o', label="Sieve of Eratosthenes and Rabin-Miller")
plt.xlabel("Prime Number Length (bits)")
plt.ylabel("Time (msec)")
plt.title("Generation of Prime Numbers of Different Lengths Using Two Algorithms")
plt.xticks(n_bits_list)
plt.legend()
plt.grid(True)
plt.show()

##### Run functional tests with the new generate_prime

Now that we've enhanced the efficiency of generating a prime number, let's execute the functional tests again to verify that the core functionality remains 
 uncompromised.

In [ ]:
generate_prime = generate_prime_miller_rabin_sieve_er
all_functional_tests()

#### 5.2.2 What is the effect of RSA key length on the private exponent $d$ generation time?

In RSA cryptography, the term "key size" refers to the length in bits of the modulus $n$. For instance, if we want a 2048-bit key size, we would need to generate two prime numbers, each approximately 1024 bits long, and then multiply them to obtain the modulus $n$.

However, the actual RSA keys used for encryption and decryption are the public exponent $e$ and the private exponent $d$, respectively. The key size (i.e., the bit length of $n$) determines the difficulty of factoring $n$ to derive the private key $d$ from the public key $e$.

Now, let's explore how the time required to generate the public/private key pair scales with the key size.

##### Explore the time to calculate the modular inverse using brute force

In [ ]:
# function `mean_time_msec` is defined in the companion notebook RSA Helper Functions.ipynb
def get_time_to_calculate_d(e, key_length, fn_mod_inverse):
    """
    Calculates the time required to compute the modular inverse 'd'
    for a given public exponent 'e' and key length.
    Args:
        e (int): The public exponent 'e'.
        key_length (int): The total length of the RSA key.
        fn_mod_inverse (function): The function to calculate the modular inverse 'd'.
    Returns:
        float: The time in milliseconds taken to compute 'd'.
    """
    prime_length = key_length // 2
    attempt = 0
    time_to_calculate_d = 1
    # sometimes the random primes lead to no modular inverse of e so we keep trying with other primes
    while attempt < 5:
        try:
            p, q = generate_prime_miller_rabin_sieve_er(prime_length), generate_prime_miller_rabin_sieve_er(prime_length)
            phi_n = eulers_totient(p, q)
            time_to_calculate_d = mean_time_msec(5)(fn_mod_inverse)(e, phi_n)
            break
        except Exception as exception:
            attempt += 1
            print(exception)
    if (attempt >= 5):
        raise Exception("After 5 tries did not find mod_inverse of {e}")

    return time_to_calculate_d

In [ ]:
e = 65537
key_lengths = [16, 18, 20, 22, 24, 26, 28]

times_to_calculate_d = [get_time_to_calculate_d(e, key_length, mod_inverse) for key_length in key_lengths]
print(*[f"Key size: {key_bits}, time to calculate d: {round(time_d, 2)} msec" for key_bits, time_d in zip(key_lengths, times_to_calculate_d)], sep='\n')

When we examine the times required to calculate the inverse, it becomes evident that there is an issue. For a key size of 28 bits, the calculation of the private exponents already takes 10+ seconds. This approach is not scalable for key sizes of 1024 and 2048 bits. 
The reason behind this poor performance is that in the section "5.1 Trivial Implementation" (🔴LINK) we implemented the calculation of the modular inverse using a brute-force method. Let's explore whether utilizing the Extended Euclidean Algorithm can enhance the performance.

##### Try to improve the calculation of modular inverse by using the Extended Euclidean Algorithm

In [ ]:
def extended_gcd(a, b):
    """The extended Euclidean algorithm."""
    if a == 0:
        return b, 0, 1
    else:
        gcd, x1, y1 = extended_gcd(b % a, a)
        x = y1 - (b // a) * x1
        y = x1
        return gcd, x, y

def mod_inverse_extended_euclidean(e, phi):
    """Compute the modular inverse of e modulo phi."""
    gcd, x, y = extended_gcd(e, phi)
    if gcd != 1:
        raise ValueError(f"No modular inverse for e={e} and phi={phi}")
    else:
        return x % phi


In [ ]:
key_lengths = [16, 18, 20, 22, 24, 26, 28]
times_to_calculate_d = [get_time_to_calculate_d(e, key_length, mod_inverse_extended_euclidean) for key_length in key_lengths]
print(*[f"Key size: {key_bits}, time to calculate d: {round(time_d, 4)} msec" for key_bits, time_d in zip(key_lengths, times_to_calculate_d)], sep='\n')

With notable progress achieved, we can now explore the utilization of more extended key lengths.

In [ ]:
e = 65537
key_lengths = [256, 512, 1024, 2048, 3072, 4096]
times_to_calculate_d = [get_time_to_calculate_d(e, key_length, mod_inverse_extended_euclidean) for key_length in key_lengths]
print(*[f"Key size: {key_bits}, time to calculate d: {round(time_d, 4)} msec" for key_bits, time_d in zip(key_lengths, times_to_calculate_d)], sep='\n')

From the output we see that while the key size increases by a factor of 16, the time to calculate $d$ increases sub-linearly only by a factor of 2. Furthermore, $d$ calculation times of a microseconds shows that the Extended Euclidean algorithm is very efficient.

##### Run functional tests with the new mod_inverse

In [ ]:
mod_inverse = mod_inverse_extended_euclidean
all_functional_tests()

#### 5.2.3 What is the impact of $e$'s length on $d$'s generation time?

Now that we know the modular inverse calculation is very fast even for 4096 bit keys we can run a combined performance test for various lengths of the public exponent $e$ and RSA key lengths. 

##### Performance test for various lengths of $e$

The value of $e$ needs to satisfy two conditions: it must be greater than 1 and less than $\varphi(n)$, and it must be coprime with $\varphi(n)$, meaning their greatest common divisor is 1. By choosing the maximum length for $e$ to be equal to that of $p$, we ensure that these conditions are met. For the performance test let's choose 4 values of $e$ depending on the prime number length (which is about one half of the RSA key length): a quarter the length of the prime, one half, tree quarters and the length of the prime.

In [ ]:
# this cell runs for ~ 55 seconds
key_lengths = [1024, 2048, 3072, 4096]

def generate_exponents(bit_length, fn_generate_prime):
    """
    Generate 🔴three prime numbers of sizes approximately 1/3, 2/3, and the full bit length of the prime.
    Args:
        bit_size (int): The desired bit size of the largest prime number.
    Returns:
        tuple: A tuple containing three prime numbers of sizes approximately 1/3, 2/3, and the full bit size.
    """

    prime1 = fn_generate_prime(bit_length // 4)      # 1/4
    prime2 = fn_generate_prime(bit_length // 2)      # 1/2
    prime3 = fn_generate_prime(3*(bit_length // 4))  # 3/4
    prime4 = fn_generate_prime(bit_length)           # 1

    return prime1, prime2, prime3, prime4

performance_per_key_length = []
for key_length in key_lengths:
    prime_length = key_length//2
    exponent_length_bits = [prime_length // 4, prime_length // 2, 3*(prime_length // 4), prime_length]
    exponents = generate_exponents(prime_length, generate_prime_miller_rabin_sieve_er)

    times_to_calculate_d = [get_time_to_calculate_d(e, key_length, mod_inverse_extended_euclidean) for e in exponents]
    performance_per_key_length.append(times_to_calculate_d)

##### Plot the results

In [ ]:
e_lengths = [1/4, 1/2, 3/4, 1]

plt.figure(figsize=(10, 6))
plt.plot(e_lengths, performance_per_key_length[0], marker='o', color="green", label=f'Key Length=1024')
plt.plot(e_lengths, performance_per_key_length[1], marker='o', color="orange", label=f'Key Length=2048')
plt.plot(e_lengths, performance_per_key_length[2], marker='o', color = "blue", label=f'Key Length=3072')
plt.plot(e_lengths, performance_per_key_length[3], marker='o', color = "purple", label=f'Key Length=4096')
plt.xlabel("Length of $e$ as fraction of key length (bits)")
plt.ylabel("Time to calculate $d$ (msec)")
plt.title('Time to Calculate $d$ for Different Key Lengths')
plt.xticks(e_lengths, ["1/4 key_length", "1/2 key_length", "3/4 key_length", "key_length"])
plt.legend()
plt.grid(True)
plt.show()

#### 5.2.4 How does the message block length affect the performance of RSA encryption and decryption?

Previously, we discussed that RSA encryption operates on block chunks (🔴LINK TO SECTION), and the size of these blocks depends on the length of $n$. However, in the trivial implementation (🔴LINK TO SECTION) we provided earlier, we divided the message into individual characters. Let's now explore how dividing the message into larger chunks would impact the time required for encryption and decryption.

##### Divide message into blocks before encrypting

In [ ]:
def string_to_ascii_number(string):
    """
    ASCII encode each character of a string
    Args:
        string (str): The string to be converted.
    Returns:
        number: The ASCII representation of the input string as a single large number.
    """
    ascii_nums = [ord(char) for char in string]
    # Convert each number to a string with leading zeros if it's a 2-digit number
    ascii_strings = [f"{num:03d}" if num < 100 else str(num) for num in ascii_nums]
    number_as_string = "".join(ascii_strings)
    return int(number_as_string)

def ascii_number_to_string(number):
    """
    Divide a number into tripples of digits and treat each tripple as an ASCII encoded character
    Args:
        number (int): The number to be converted.
    Returns:
        str: The corresponding string representation of the number.
    """
    number_as_string = str(number)
    if (len(number_as_string) % 3 != 0):
        number_as_string = "0" + number_as_string
    # Split number_as_string into groups of 3 characters
    ascii_chars = [number_as_string[i:i+3] for i in range(0, len(number_as_string), 3)]
    chars = [chr(int(char)) for char in ascii_chars]
    return "".join(chars)

def number_of_bytes(n):
    """Calculates the number of bytes required to represent an integer `n`"""
    if n == 0:
        return 1  # Even 0 takes up one byte
    bit_length = n.bit_length()
    byte_length = (bit_length + 7) // 8  # Divide by 8 and round up
    return byte_length

def message_to_blocks(message, block_length):
    """Split a string into blocks of specified length"""
    return [message[i:i + block_length] for i in range(0, len(message), block_length)]

def encrypt_block(block, e, n):
    """Encrypts a single block of text using RSA encryption"""
    ascii_block = string_to_ascii_number(block)
    return pow(ascii_block, e, n)

def decrypt_block(encrypted_block, d, n):
    """Decrypts a single block of text using RSA encryption"""
    ascii_block = pow(encrypted_block, d, n)
    return ascii_number_to_string(ascii_block)

def message_to_encrypted_blocks(message, e, n):
    """
    Encrypts a message by splitting it into blocks and encrypting
    each block using RSA encryption.
    Args:
        message (str): The message to be encrypted.
        e (int): The public exponent used for encryption.
        n (int): The modulus used for encryption.
    Returns:
        list: A list of encrypted blocks (integers).
    """
    block_length_bytes = number_of_bytes(n) // 2
    blocks = message_to_blocks(message, block_length_bytes)
    encrypted_blocks = [encrypt_block(block, e, n) for block in blocks]
    return encrypted_blocks

def encrypted_blocks_to_message(encrypted_blocks, d, n):
    """
    Decrypts encrypted blocks using RSA decryption and concatenates
    them into the original message.
    Args:
        encrypted_blocks (list): A list of encrypted blocks (integers).
        d (int): The private exponent used for decryption.
        n (int): The modulus used for decryption.
    Returns:
        str: The decrypted original message.
    """
    decrypted_blocks = [decrypt_block(block, d, n) for block in encrypted_blocks]
    decrypted_message = "".join(decrypted_blocks)
    return decrypted_message

In [ ]:
long_text = Path("test_data/a_tale_of_two_cities_760000_chars.txt").read_text()
message = long_text[:10000]

e, d, n = generate_keys(prime_length_bits = 1024)
encrypted_message = message_to_encrypted_blocks(message, e, n)
decrypted_message = encrypted_blocks_to_message(encrypted_message, d, n)

assert(message == decrypted_message)

##### Performance test for encrypting/decrypting messages of various lengths

In [ ]:
# this cell runs for ~1min
long_text = Path("test_data/a_tale_of_two_cities_760000_chars.txt").read_text()
e, d, n = generate_keys(prime_length_bits = 1024)
message_lengths = [1000, 2000, 3000, 5000, 10000, 20000]
input_messages = [long_text[:n] for n in message_lengths]
encrypted_messages = [message_to_encrypted_blocks(message, e, n) for message in input_messages]

times_to_encrypt = [mean_time_msec(3)(message_to_encrypted_blocks)(message, e, n) for message in input_messages]
times_to_decrypt = [mean_time_msec(3)(encrypted_blocks_to_message)(encrypted_message, d, n) for encrypted_message in encrypted_messages]

print(times_to_encrypt)
print(times_to_decrypt)

##### Plot the results

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(message_lengths, times_to_encrypt, marker='o', label='Encryption Time')
plt.plot(message_lengths, times_to_decrypt, marker='o', label='Decryption Time')

plt.xlabel("Message Length (characters)")
plt.ylabel("Time (msec)")
plt.title("Encryption and Decryption Times for Different Message Lengths")
plt.xticks(message_lengths)
plt.legend()
plt.grid(True)
plt.show()

The graph illustrates that the time required for encryption and decryption does not differ. Additionally, the plot shows a linear increase in the time taken as the length of the input message increases. Moving forward, we can incorporate the size of the cryptographic key as an additional variable in the performance testing to analyze how it impacts the encryption and decryption times.

#### 5.2.5 How does key size impact encryption and decryption times?

##### Performance test

In [ ]:
message = long_text[:10000]
key_lengths = [256, 512, 1024, 2048]

# list of tuples (e, d, n)
rsa_params = [generate_keys(prime_length_bits = key_length // 2) for key_length in key_lengths]
encrypted_messages = [message_to_encrypted_blocks(message, e, n) for (e, d, n) in rsa_params]
times_to_encrypt = [mean_time_msec(1)(message_to_encrypted_blocks)(message, e, n) for (e, d, n) in rsa_params]
times_to_decrypt = [mean_time_msec(1)(encrypted_blocks_to_message)(encrypted_message, d, n) for encrypted_message, (e, d, n) in zip(encrypted_messages, rsa_params)]

print(times_to_encrypt)
print(times_to_decrypt)

##### Plot the results

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(key_lengths, times_to_encrypt, marker='o', label='Encryption Time')
plt.plot(key_lengths, times_to_decrypt, marker='o', label='Decryption Time')
plt.xlabel("Key Length (bits)")
plt.ylabel("Time (msec)")
plt.title("Encryption and Decryption Times for Different Key Lengths")
plt.xticks(key_lengths)
plt.legend()
plt.grid(True)
plt.show()

The data shows that as the length of the cryptographic key becomes larger, the time required for the encryption process increases exponentially. Additionally, consistent with the previous graph, we observe that the times taken for encryption and decryption operations are equivalent.

##### Functional tests with the new encrypt/decrypt functions

After modifying the encryption and decryption functions to handle message segmentation into blocks, we need to execute the functional tests to verify that the whole RSA existing functionality still works as expected.

In [ ]:
encrypt_message_to_cipher = message_to_encrypted_blocks
decrypt_cipher_to_message = encrypted_blocks_to_message
all_functional_tests()